### Notebook for working on text distance things.

Word Mover's Distance

In [139]:
import gensim.downloader as api
import pandas as pd
import numpy as np
import ipdb
from itertools import combinations

In [3]:
word_vectors = api.load("glove-wiki-gigaword-100")

In [4]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

similarity = word_vectors.wmdistance(sentence_obama, sentence_president)

In [6]:
# read in topics csv, ignore last 5 rows.

topics = pd.read_csv('peloton_topics.csv', index_col = 'Brand', nrows = 5) 

In [7]:
topics

,Peloton,MIRROR,Tonal,FightCamp,Tempo,Echelon,Nordic Track,Life Fitness
Brand,,,,,,,,
Topic 1,Get 150$ Off accessories with the Peloton bike,The Nearly Invisible Home Gym,The Smartest Home Gym,Connected at-home boxing,Personalized guidance with every workout,"Home fitness, redefined.",Experience the Difference of Interactive Perso...,Explore our Top Sellers in Cardio Equipment.
Topic 2,Try classess free on the peloton bike,Hiding In Plain Sight,The Future Of Fitness,Everything you need to fight for the best vers...,Meet the Tempo Studio,Get 15% off ann EX-3 For Mother's Day,0% Apr for 36 Months,The World's Most Trusted Fitness Brands
Topic 3,Unlock Cardio + More with the Peloton Bike +,One Mirror. Every Workout.,"Get Stronger, Faster",Addicted After Just One Punch,Smarter tracking. Better results.,Biking Revolutionized,Experience Automatic Trainer Control,Top Sellers. Something for Everyone.
Topic 4,Tap into motivation whenever you want it,"Train Now, Pay Over Time",An Entire Gym In Your Home,Why not train like a fighter?,Fitness made to fit anywhere.,"Immersive, studio-quality cycling with a globa...",See the World from the Comfort of Home,Turn Up the Intensity in Your Facility
Topic 5,"Find the workout that fits your mood, goals, m...",Advanced Tracking Technology,"Artificial Intelligence, Real Results",We know life comes first,Personalized classes designed for you. Whoever...,A personal trainer and fitness studio in one s...,Where High-Energy Workouts Meet Low-Impact Tra...,Introducing Life Fitness Axiom Series


In [8]:
word_vectors.wmdistance(topics.iloc[1,1].lower().split(), topics.iloc[1,2].lower().split()) #example

5.60379484716475

In [9]:
# create a quick wmd function

def wmdcalc(x, y) :
    
    return(word_vectors.wmdistance(x.lower().split(), y.lower().split()))
    

In [143]:
# what we want to do is compute pairwise word distances for all topics in a brand; 
# and then compute pairwise distances for all brands (by turning their topics into a single string)

for brand in topics.columns :
    
    topicsList = topics[brand].tolist()
    x = list(combinations(topicsList,2))# use itertools to get all pairwise combinations
    dist = []
    for combo in x :
        dist.append(wmdcalc(combo[0],combo[1]))
    print(brand, np.mean(dist), np.std(dist))

Peloton 4.33604814318066 0.8530265766871147
MIRROR 5.589588958135106 0.6577011934404708
Tonal 5.506082698236428 0.8333959908515121
FightCamp 5.280788973328589 0.9213599174406648
Tempo 5.4609168009208515 0.47510194912599146
Echelon 5.979602728955562 0.9097750436164916
Nordic Track 5.567719760680762 0.6661672927847914
Life Fitness 5.243515826652176 0.5136382051200494


In [163]:
# Now compute pairwise WMD between a single corpus of all topics for each brand.

distDF = pd.DataFrame(columns = topics.columns, index=topics.columns)

for idx,brand1 in enumerate(distDF.columns.values) :
    for jdx,brand2 in enumerate(distDF.index.values) :
        brand1Content = topics[brand1].str.cat(sep=' ')
        brand2Content = topics[brand2].str.cat(sep=' ')
        distDF.iloc[jdx,idx] = wmdcalc(brand1Content, brand2Content)

In [164]:
distDF

,Peloton,MIRROR,Tonal,FightCamp,Tempo,Echelon,Nordic Track,Life Fitness
Peloton,0,4.87595,4.62772,4.68737,4.55483,4.41311,4.67954,4.55684
MIRROR,4.87595,0,4.15965,4.53012,4.80637,4.43947,4.66905,4.72411
Tonal,4.62772,4.15965,0,4.61182,4.57842,4.12485,4.31117,4.10012
FightCamp,4.68737,4.53012,4.61182,0,4.38671,4.28121,4.48852,4.45381
Tempo,4.55483,4.80637,4.57842,4.38671,0,4.37525,4.56684,4.47478
Echelon,4.41311,4.43947,4.12485,4.28121,4.37525,0,3.7698,4.45919
Nordic Track,4.67954,4.66905,4.31117,4.48852,4.56684,3.7698,0,4.6154
Life Fitness,4.55684,4.72411,4.10012,4.45381,4.47478,4.45919,4.6154,0


In [171]:
topics[['MIRROR','Peloton']]

,MIRROR,Peloton
Brand,,
Topic 1,The Nearly Invisible Home Gym,Get 150$ Off accessories with the Peloton bike
Topic 2,Hiding In Plain Sight,Try classess free on the peloton bike
Topic 3,One Mirror. Every Workout.,Unlock Cardio + More with the Peloton Bike +
Topic 4,"Train Now, Pay Over Time",Tap into motivation whenever you want it
Topic 5,Advanced Tracking Technology,"Find the workout that fits your mood, goals, m..."
